# Data Mining Project
### Giorgio Donati, g.donati24@studenti.unipi.it
### Pietro Francaviglia, p.francaviglia1@studenti.unipi.it
#### A.Y. 2021-2022


## Libraries and datasets

In [52]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import itertools
from sklearn import tree
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from t3_constants import *
from t3_utility import *

In [53]:
with open(f'{PICKLE_FOLDER}{LABELLED}{USER_DF}.pkl', 'rb') as f:
    u_df = pickle.load(f)

with open(f'{PICKLE_FOLDER}{PURE}{LABELLED}{USER_DF}.pkl', 'rb') as f:
    pure_u_df = pickle.load(f)

In [54]:
u_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 795 entries, 1 to 823
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   item_count            795 non-null    int64  
 1   item_dist_count       795 non-null    int64  
 2   max_items_per_b       795 non-null    int64  
 3   price_entropy         795 non-null    float64
 4   max_item_dist_per_b   795 non-null    int64  
 5   total_price           795 non-null    float64
 6   basket_count          795 non-null    int64  
 7   category_count        795 non-null    int64  
 8   shop_count            795 non-null    int64  
 9   avg_items_per_basket  795 non-null    float64
 10  avg_baskets_per_d     795 non-null    float64
 11  label                 754 non-null    object 
dtypes: float64(4), int64(7), object(1)
memory usage: 80.7+ KB


In [55]:
train_x, test_x, train_y, test_y = train_test_split(
    u_df[u_df[LAB].notna()].drop(columns=[LAB]),
    u_df[u_df[LAB].notna()][LAB],
    test_size=0.2,
    stratify=u_df[u_df[LAB].notna()][LAB])
x = train_x.values
y = train_y.values.ravel().astype(int)
y

array([2, 2, 5, 1, 1, 1, 3, 1, 0, 3, 3, 4, 4, 7, 5, 5, 4, 4, 8, 0, 3, 3,
       0, 1, 1, 0, 9, 3, 0, 4, 6, 1, 1, 3, 2, 2, 2, 0, 2, 4, 0, 1, 1, 0,
       0, 0, 2, 2, 2, 4, 1, 7, 2, 2, 3, 1, 6, 1, 2, 3, 0, 5, 0, 5, 7, 2,
       2, 2, 0, 0, 0, 7, 1, 3, 1, 5, 1, 3, 2, 0, 5, 3, 8, 4, 4, 1, 2, 3,
       0, 2, 5, 2, 2, 4, 0, 3, 6, 5, 8, 3, 0, 7, 3, 2, 1, 4, 1, 5, 3, 5,
       0, 3, 3, 1, 5, 7, 4, 0, 3, 4, 4, 2, 5, 6, 1, 3, 0, 0, 5, 5, 1, 2,
       3, 2, 3, 1, 5, 4, 2, 5, 5, 2, 2, 0, 5, 1, 5, 3, 5, 2, 3, 6, 2, 4,
       1, 1, 0, 1, 0, 1, 5, 1, 4, 4, 3, 5, 2, 0, 1, 1, 3, 6, 3, 2, 5, 3,
       5, 5, 4, 3, 0, 5, 2, 3, 1, 9, 3, 5, 0, 1, 1, 0, 4, 3, 0, 2, 0, 0,
       3, 1, 1, 5, 1, 1, 3, 5, 3, 0, 3, 4, 5, 1, 5, 4, 4, 0, 6, 1, 1, 2,
       0, 2, 4, 5, 3, 4, 5, 0, 4, 1, 3, 2, 4, 5, 4, 2, 5, 1, 1, 4, 2, 0,
       7, 2, 2, 0, 3, 3, 2, 2, 3, 0, 3, 6, 0, 2, 1, 4, 2, 2, 1, 0, 1, 0,
       5, 4, 4, 5, 2, 1, 2, 4, 1, 2, 1, 4, 4, 4, 1, 6, 2, 5, 2, 0, 0, 3,
       2, 4, 0, 4, 5, 1, 2, 5, 4, 5, 4, 0, 1, 4, 1,

In [56]:
def cross_validation(model, X: np.ndarray, y: np.ndarray, n_splits: int) -> np.ndarray:
    """Return validation scores across the k folds of cross-validation."""
    skf = StratifiedKFold(n_splits=n_splits, random_state=0, shuffle=True)
    val_score = []
    for train_index, test_index in skf.split(X, y):
        model.fit(X[train_index], y[train_index].ravel())
        val_score.append(model.score(X[test_index], y[test_index].ravel() + 1))
    return np.array(val_score)

def cross_validation_summary(model, X: np.ndarray, y: np.ndarray, n_splits: int) -> np.ndarray:
    """Returns validation accuracy score of model (mean and std over all the splits)."""
    val_score = cross_validation(model, X, y, n_splits)
    return val_score.mean(), val_score.std()


In [57]:
## Hyper-parameters grid search for Decision Tree

hyp_dict = {
    'criterion': ['gini', 'entropy'],
    'min_impurity_decrease': [0, 1, 1e-1, 1e-2, 1e-3, 1e-4],
    'min_samples_leaf': [1, 3, 5]
}
combinations = itertools.product(*(hyp_dict[key] for key in sorted(hyp_dict)))

results = []

for criterion, min_impurity_decrease, min_samples_leaf in combinations:
    model = tree.DecisionTreeClassifier(
        criterion=criterion,
        min_impurity_decrease=min_impurity_decrease,
        min_samples_leaf=min_samples_leaf,
        random_state=0
    )
    mean_val_score, std_val_score = cross_validation_summary(model, x, y, 5)
    res = criterion, min_impurity_decrease, min_samples_leaf, mean_val_score, std_val_score
    results.append(res)

decision_tree_results_df = pd.DataFrame(results, columns=['criterion', 'min_impurity_decrease', 'min_samples_leaf', 'mean_val_score', 'std_val_score'])
decision_tree_results_df.sort_values('mean_val_score').tail(1)

/Users/hdjdox/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/hdjdox/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/hdjdox/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/hdjdox/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(
/Users/hdjdox/Library/Python/3.8/lib/python/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 4 members, whi

,criterion,min_impurity_decrease,min_samples_leaf,mean_val_score,std_val_score
12,gini,0.001,1,0.139284,0.019799
